In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [55]:
df = pd.read_csv('data_concat_10y.csv')

# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['city', 'date'])

soil = ['soil_moisture_7_to_28cm_mean',
                'soil_moisture_0_to_7cm_mean', 'soil_temperature_0_to_7cm_mean',
                'soil_temperature_7_to_28cm_mean',]

# Features and target variables
feature_cols = ['temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean',
                'cloud_cover_mean', 'relative_humidity_2m_mean',  'rain_sum', 'precipitation_hours',
                'daylight_duration', 'snowfall_sum',
                'surface_pressure_mean', 'surface_pressure_max', 'surface_pressure_min',
                 'et0_fao_evapotranspiration_sum']

# Target columns (variables to predict for next 2 days)
target_cols = ['temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean',
               'soil_moisture_7_to_28cm_mean',
               'soil_moisture_0_to_7cm_mean', 'soil_temperature_0_to_7cm_mean',
               'soil_temperature_7_to_28cm_mean', 'rain_sum', 'precipitation_hours',
               'daylight_duration', 'snowfall_sum', 'surface_pressure_mean',
               'et0_fao_evapotranspiration_sum']

# One-hot encode the 'city' column
enc = OneHotEncoder(sparse_output=False)
city_encoded = enc.fit_transform(df[['city']])
city_encoded_df = pd.DataFrame(city_encoded, columns=enc.get_feature_names_out(['city']))
df = pd.concat([df, city_encoded_df], axis=1)


In [56]:
df

,date,temperature_2m_max,temperature_2m_min,temperature_2m_mean,cloud_cover_mean,relative_humidity_2m_mean,soil_moisture_7_to_28cm_mean,soil_moisture_0_to_7cm_mean,soil_temperature_0_to_7cm_mean,soil_temperature_7_to_28cm_mean,...,city_Krakow,city_Lodz,city_Lublin,city_Olsztyn,city_Poznan,city_Rzeszow,city_Szczecin,city_Warszawa,city_Wroclaw,city_Zielona Gora
25578,2015-05-13 22:00:00+00:00,15.1280,6.0780,10.888416,70.583336,72.722374,0.281208,0.277542,12.032167,11.682167,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25579,2015-05-14 22:00:00+00:00,14.5280,6.1280,9.490500,53.791668,72.827540,0.277625,0.283167,10.888417,11.107167,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25580,2015-05-15 22:00:00+00:00,14.5280,3.1780,9.636332,37.750000,60.329510,0.276083,0.286917,10.228001,10.230083,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25581,2015-05-16 22:00:00+00:00,13.2780,6.9780,10.469667,55.958332,65.312090,0.272458,0.274667,10.811334,10.498834,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25582,2015-05-17 22:00:00+00:00,14.1280,6.0280,10.136334,33.625000,58.688797,0.268333,0.267292,10.307166,10.240500,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40189,2025-05-09 22:00:00+00:00,15.6945,9.3945,12.248666,91.583336,61.886353,0.111208,0.069167,12.246585,11.292417,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
40190,2025-05-10 22:00:00+00:00,14.9945,4.0445,10.407001,22.750000,57.747380,0.107792,0.082167,12.079915,11.569501,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
40191,2025-05-11 22:00:00+00:00,15.4445,3.5945,10.454917,31.000000,48.354160,0.101167,0.059833,11.415334,11.469501,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
40192,2025-05-12 22:00:00+00:00,19.3945,4.1445,12.738250,28.333334,43.718700,0.099708,0.064042,13.332000,11.769501,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [57]:
df['cloud_cover_mean'] = df['cloud_cover_mean'] / 10
df['relative_humidity_2m_mean'] = df['relative_humidity_2m_mean'] / 10
df['soil_moisture_7_to_28cm_mean'] = df['soil_moisture_7_to_28cm_mean'] * 10
df['soil_moisture_0_to_7cm_mean'] = df['soil_moisture_0_to_7cm_mean'] * 10
df['daylight_duration'] = df['daylight_duration'] / 3600
df['surface_pressure_mean'] = df['surface_pressure_mean'] / 100
df['surface_pressure_max'] = df['surface_pressure_max'] / 100
df['surface_pressure_min'] = df['surface_pressure_min'] / 100


In [58]:
df[target_cols]

,temperature_2m_max,temperature_2m_min,temperature_2m_mean,soil_moisture_7_to_28cm_mean,soil_moisture_0_to_7cm_mean,soil_temperature_0_to_7cm_mean,soil_temperature_7_to_28cm_mean,rain_sum,precipitation_hours,daylight_duration,snowfall_sum,surface_pressure_mean,et0_fao_evapotranspiration_sum
25578,15.1280,6.0780,10.888416,2.812083,2.775417,12.032167,11.682167,0.7,5.0,15.772811,0.0,9.918209,2.883074
25579,14.5280,6.1280,9.490500,2.776250,2.831667,10.888417,11.107167,2.6,4.0,15.827300,0.0,9.940878,2.669162
25580,14.5280,3.1780,9.636332,2.760834,2.869166,10.228001,10.230083,0.2,1.0,15.880767,0.0,10.000511,3.248107
25581,13.2780,6.9780,10.469667,2.724583,2.746666,10.811334,10.498834,0.9,3.0,15.933139,0.0,9.981413,3.101965
25582,14.1280,6.0280,10.136334,2.683333,2.672917,10.307166,10.240500,0.0,0.0,15.984342,0.0,9.973179,3.337464
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40189,15.6945,9.3945,12.248666,1.112083,0.691666,12.246585,11.292417,1.0,4.0,15.418725,0.0,10.022637,2.635017
40190,14.9945,4.0445,10.407001,1.077917,0.821667,12.079915,11.569501,0.0,0.0,15.472465,0.0,10.022349,3.976310
40191,15.4445,3.5945,10.454917,1.011667,0.598333,11.415334,11.469501,0.0,0.0,15.525671,0.0,10.011889,3.671435
40192,19.3945,4.1445,12.738250,0.997083,0.640417,13.332000,11.769501,0.0,0.0,15.578282,0.0,10.022142,4.631162


In [59]:
def create_sequences(data, city_data, days_input=7, days_output=4):
    X, y = [], []
    cities = data['city'].unique()

    for city in cities:
        city_df = data[data['city'] == city]
        if len(city_df) < days_input + days_output:
            continue

        city_features = city_df[feature_cols].values
        city_targets = city_df[target_cols].values
        city_encoded = city_data[data['city'] == city].iloc[0].values

        for i in range(len(city_df) - days_input - days_output + 1):
            # Get 3 days of input features
            input_seq = city_features[i:i + days_input, :]
            # Flatten the input sequence
            flat_input = input_seq.flatten()
            # Add city encoding
            input_with_city = np.concatenate([flat_input, city_encoded])
            X.append(input_with_city)

            # Get 2 days of output targets
            output_seq = city_targets[i + days_input:i + days_input + days_output, :]
            # Flatten the output sequence
            flat_output = output_seq.flatten()
            y.append(flat_output)

    return np.array(X), np.array(y)

In [60]:
# Get city encoded data
city_data = df[df.columns[df.columns.str.startswith('city_')]]

# Create sequences
X, y = create_sequences(df, city_data)

# Split data into train (80%), validation (10%), and test (10%) sets
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5,
                                                  random_state=42)

In [61]:
X_val

array([[21.122002 , 13.072    , 17.417835 , ...,  0.       ,  0.       ,
         0.       ],
       [25.887    , 15.887    , 20.839083 , ...,  0.       ,  0.       ,
         0.       ],
       [13.028    ,  8.978    , 10.634252 , ...,  0.       ,  0.       ,
         0.       ],
       ...,
       [ 6.087    ,  0.187    ,  2.8703334, ...,  0.       ,  0.       ,
         0.       ],
       [22.583    ,  9.533    , 15.566334 , ...,  0.       ,  0.       ,
         0.       ],
       [ 5.687    ,  2.837    ,  4.934916 , ...,  0.       ,  0.       ,
         0.       ]])

In [62]:
X_val[0]

array([21.122002  , 13.072     , 17.417835  ,  7.4625    ,  7.347927  ,
        0.        ,  0.        , 16.50955611,  0.        , 10.064709  ,
       10.0698346 , 10.058026  ,  2.9271076 , 23.872002  , 11.672     ,
       18.799082  ,  5.0666668 ,  7.291673  ,  0.1       ,  1.        ,
       16.49640556,  0.        , 10.057651  , 10.064673  , 10.049621  ,
        4.062164  , 25.122002  , 13.122     , 19.840752  ,  1.2291667 ,
        6.665441  ,  0.        ,  0.        , 16.48153056,  0.        ,
       10.0797363 , 10.0921356 , 10.064805  ,  4.5394077 , 26.972     ,
       14.172     , 21.397001  ,  0.15833334,  5.637807  ,  0.        ,
        0.        , 16.46497944,  0.        , 10.1046045 , 10.114021  ,
       10.0945447 ,  5.7145147 , 29.172     , 16.472     , 23.380331  ,
        0.        ,  4.889261  ,  0.        ,  0.        , 16.4467925 ,
        0.        , 10.1164923 , 10.128086  , 10.102255  ,  6.478379  ,
       32.122     , 17.772001  , 25.613665  ,  0.3625    ,  5.18

In [63]:
test1_X = X_val[0]
test1_y = y_val[0]

In [64]:
y_val

array([[27.072     , 20.222     , 24.530333  , ...,  0.        ,
         9.981128  ,  4.7224426 ],
       [28.187     , 17.737     , 22.699501  , ...,  0.        ,
        10.1884064 ,  4.724208  ],
       [26.377998  , 16.528     , 21.792585  , ...,  0.        ,
        10.091992  ,  5.6124353 ],
       ...,
       [17.337     , 11.137     , 14.414083  , ...,  0.        ,
        10.0488086 ,  5.2765255 ],
       [12.333     ,  7.533     ,  9.8205    , ...,  0.        ,
        10.093787  ,  2.3707871 ],
       [ 3.487     ,  0.287     ,  2.14325   , ...,  0.49      ,
         9.8370557 ,  0.23486239]])

In [65]:
y_train.shape[1], X_train.shape[1]

(52, 103)

In [66]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),            
    tf.keras.layers.Dense(128, activation='relu'), 
    tf.keras.layers.Dense(256, activation='relu'), 
    tf.keras.layers.Dense(64, activation='relu'),  
    tf.keras.layers.Dense(y_train.shape[1])                     
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [67]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [68]:
history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/200
547/547 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 15.9825 - mae: 2.4458 - val_loss: 6.1657 - val_mae: 1.5191
Epoch 2/200
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 6.2312 - mae: 1.5286 - val_loss: 6.0572 - val_mae: 1.4688
Epoch 3/200
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 6.1325 - mae: 1.4867 - val_loss: 5.9236 - val_mae: 1.4316
Epoch 4/200
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 5.9211 - mae: 1.4416 - val_loss: 5.7981 - val_mae: 1.4169
Epoch 5/200
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 5.9747 - mae: 1.4343 - val_loss: 5.8379 - val_mae: 1.4155
Epoch 6/200
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 5.7823 - mae: 1.4058 - val_loss: 5.8036 - val_mae: 1.4180
Epoch 7/200
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 5.7771 - mae: 1.4068 - val_loss: 6.0874 - val_mae: 1.4684
Epoch 8/200
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 5.7446 - mae: 1.4035 - val_loss: 5.6773 - val_mae: 1.3840
Epoch 9/200
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms

In [69]:
%%time
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test MAE: {test_mae}")

137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.9683 - mae: 1.2594
Test Loss: 4.967806816101074
Test MAE: 1.2576626539230347
CPU times: total: 422 ms
Wall time: 327 ms


In [70]:
prediction = model.predict(X_test[10][None, :])
prediction = prediction.reshape(4, len(target_cols))
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step


array([[ 7.379914  ,  1.5167068 ,  4.302701  ,  2.840133  ,  2.975495  ,
         3.8460226 ,  3.5041134 ,  3.0230474 ,  8.293677  , 11.309841  ,
         0.3872465 , 10.346354  ,  0.8387297 ],
       [ 8.366096  ,  1.502107  ,  4.7775536 ,  2.8668096 ,  2.9243715 ,
         4.1970415 ,  3.6378727 ,  2.4016411 ,  6.699407  , 11.340894  ,
         0.250261  , 10.309498  ,  1.0757129 ],
       [ 9.215165  ,  1.9868153 ,  5.3669744 ,  2.8820057 ,  2.8886416 ,
         4.6141953 ,  4.04408   ,  1.8729395 ,  5.531002  , 11.397062  ,
         0.12450141, 10.295766  ,  1.1518526 ],
       [ 9.3446455 ,  1.7718618 ,  5.4173155 ,  2.8636045 ,  2.892037  ,
         4.8277526 ,  4.4595327 ,  1.4639177 ,  4.018456  , 11.482013  ,
         0.04140882, 10.327455  ,  1.2454181 ]], dtype=float32)

In [71]:
expected = y_test[10].reshape(4, len(target_cols))
expected

array([[ 7.6194997 ,  4.1695    ,  5.232     ,  2.5516668 ,  2.7966666 ,
         5.3445    ,  4.829916  ,  6.7       , 23.        , 11.25579528,
         0.        , 10.0334717 ,  0.54409415],
       [ 8.8695    ,  2.9695    ,  5.2695    ,  2.6816666 ,  2.702917  ,
         5.6736665 ,  5.384083  ,  0.        ,  0.        , 11.32951833,
         0.        , 10.1471246 ,  1.3969238 ],
       [ 9.169499  ,  1.9695001 ,  5.459082  ,  2.614583  ,  2.5237507 ,
         5.2257495 ,  5.223666  ,  0.2       ,  2.        , 11.40311944,
         0.        , 10.103022  ,  1.3344159 ],
       [10.7195    ,  3.9695    ,  7.3215833 ,  2.5749996 ,  2.5574997 ,
         6.6174164 ,  5.82575   ,  2.2       , 11.        , 11.476555  ,
         0.        , 10.049961  ,  1.7088413 ]])

In [72]:
d = ['temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean', 'soil_moisture_7_to_28cm_mean', 'soil_moisture_0_to_7cm_mean', 'soil_temperature_0_to_7cm_mean', 'soil_temperature_7_to_28cm_mean', 'rain_sum', 'precipitation_hours', 'daylight_duration', 'snowfall_sum', 'surface_pressure_mean', 'et0_fao_evapotranspiration_sum']

In [73]:
model.save("model_no_soil.keras")

In [74]:
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

In [75]:
# Scale features for SVM (important for kernel-based models)
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled = scaler_X.transform(X_val)
X_test_scaled = scaler_X.transform(X_test)

# Scale targets (optional but recommended for regression with different magnitudes)
y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled = scaler_y.transform(y_val)
y_test_scaled = scaler_y.transform(y_test)

In [78]:
X_train_sample = X_train_scaled[:4000]
y_train_sample = y_train_scaled[:4000]

In [79]:
%%time
# Define the Support Vector Regression model
svm_base = SVR(kernel='rbf', C=10, epsilon=0.1)

# Wrap it for multi-output regression
model = MultiOutputRegressor(svm_base, n_jobs=-1)

# Train the model
model.fit(X_train_sample, y_train_sample)

CPU times: total: 672 ms
Wall time: 35.4 s


MultiOutputRegressor(estimator=SVR(C=10), n_jobs=-1)

In [80]:
%%time
# Evaluate on test set
y_pred_scaled = model.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

CPU times: total: 906 ms
Wall time: 39.3 s


In [81]:
# Compute metrics
test_mse = mean_squared_error(y_test, y_pred)
test_mae = mean_absolute_error(y_test, y_pred)

print(f"Test MSE: {test_mse:.4f}")
print(f"Test MAE: {test_mae:.4f}")

Test MSE: 6.7876
Test MAE: 1.4479


In [82]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [83]:
# Define the base Random Forest Regressor
rf_base = RandomForestRegressor(
    n_estimators=20,   # Number of trees
    max_depth=None,     # Let trees expand until all leaves are pure or contain min_samples_split
    random_state=42,
    n_jobs=-1,
    verbose=1
)

In [88]:
X_train_sample = X_train[:3000]
y_train_sample = y_train[:3000]

In [89]:
%%time
# Wrap it for multi-output regression
model = MultiOutputRegressor(rf_base, n_jobs=-1)

# Train the model
model.fit(X_train_sample, y_train_sample)

CPU times: total: 875 ms
Wall time: 1min 12s


MultiOutputRegressor(estimator=RandomForestRegressor(n_estimators=20, n_jobs=-1,
                                                     random_state=42,
                                                     verbose=1),
                     n_jobs=-1)

In [90]:
%%time
# Predict on the test set
y_pred = model.predict(X_test)

CPU times: total: 2.06 s
Wall time: 2.32 s


In [91]:
# Compute evaluation metrics
test_mse = mean_squared_error(y_test, y_pred)
test_mae = mean_absolute_error(y_test, y_pred)

print(f"Test MSE: {test_mse:.4f}")
print(f"Test MAE: {test_mae:.4f}")

Test MSE: 6.0498
Test MAE: 1.3710
